In [1]:
import pyupbit
import time
from datetime import datetime, timedelta
from pytz import timezone, utc
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import trange, notebook

pd.options.display.max_rows = 1440

KST = timezone('Asia/Seoul')
date = KST.localize(datetime.now())
yesterday = date - timedelta(1)
yesterday = yesterday.strftime("%Y-%m-%d")
today = (KST.localize(datetime.now())).strftime("%Y-%m-%d")


def get_rate(p1, p2):
    return round(((p1/p2 - 1) * 100), 2)

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

def get_pick_coins(date):
    coins = pyupbit.get_tickers(fiat="KRW")
    coins_rate = []
    for coin in coins:
        coin_price = pyupbit.get_ohlcv(coin, date, count=1)
        time.sleep(0.1)
        open_price = coin_price.open.values[0]
        close_price = coin_price.close.values[0]
        coin_rate = get_rate(close_price, open_price)
        coins_rate.append([date, coin, coin_rate, ""])

    picked_coins = sorted(coins_rate, key = lambda x : -x[2])[0:5]
    return picked_coins


In [ ]:
columns = ["날짜", "코인명","전날 등락률", "1분", "2분", "3분", "4분", "5분", "고가", "종가"]

# df = pd.DataFrame(data, columns=columns)
df.to_csv('price.csv')



In [ ]:
df = pd.read_csv("price.csv")


In [ ]:
df['1분'] = np.where(df['1분'] < -3, -3, df['1분'])
df['2분'] = np.where(df['2분'] < -3, -3, df['2분'])
df['3분'] = np.where(df['3분'] < -3, -3, df['3분'])
df['4분'] = np.where(df['4분'] < -3, -3, df['4분'])
df['5분'] = np.where(df['5분'] < -3, -3, df['5분'])
df['종가'] = np.where(df['종가'] < -3, -3, df['종가'])
df['수정종가'] = np.where(((df['1분'] > -3)&(df['2분'] > -3)&(df['3분'] > -3)&(df['4분'] > -3)&(df['5분'] > -3)), df['종가'], -3)
df['고가'] = np.where(df['고가'] < -3, -3, df['고가'])
df['수정고가'] = np.where(((df['1분'] > -3)&(df['2분'] > -3)&(df['3분'] > -3)&(df['4분'] > -3)&(df['5분'] > -3)), df['고가'], 0)

In [ ]:
plt.figure(figsize = (20, 10))
plt.grid()
plt.plot(df.index, df['1분'], color='b')
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.grid()
plt.plot(df.index, df['5분'], color='r')
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.grid()
plt.plot(df.index, df['종가'], color="g")
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.grid()
plt.plot(df.index, df['고가'], color="g")
plt.show()

In [ ]:
plt.figure(figsize = (20, 10))
plt.grid()
plt.plot(df.index, df['수정고가'], color="g")
plt.show()

In [ ]:
# 매도 알고리즘
##### 폭락장에서는 사지 않는다. 

# -3퍼 아래 손절
# 저가 low가 -3퍼 아래 손절
# 저가가 고가보다 빠른 시간에 형성될 경우 -3퍼로 계산
# 경우1: -3퍼가 형성되기 이전에 10% 일 경우 절반 5퍼 + 50% 넘어갈 경우 전량매도 -> 37퍼
# 경우2: -3퍼가 형성되기 이전에 10% 절반 익절 후 0퍼 수익률로 갔을 경우 -> 5퍼
# 경우3: -3퍼까지 가지 않았으며 10% 아래 + 수익률일 경우 -> 그날의 종가 수익률 -> 5퍼 + 5.5 * 종가 수익률 
# 경우4: -3퍼가 형성되기 이전에 10% 절반 익절 후 0퍼 수익률로 가지 않을 경우 -> 그날의 종가 수익률


# 경우5(고려 사항): 특정 시간 이후로 -3%까지는 아니지만 - 수익률일 경우 절반 손절
5 - 3 => 2
coin_data 5 
110 -> 55 -> 47

In [30]:
coin_cnt = 0
coin_analysis = [] # 날짜, 코인명, 수익률 
job_coin = data
fff = coin_data

for idx, date in enumerate(dates):
    for i in range(5): # 5개의 코인 하루치 데이터 탐색
        name = fff[idx][i][1]
        data = [date, name, 0]
        check = ""
        for j in range(1440):
            if get_rate(job_coin[idx][j]["low"], job_coin[idx][1440 * i]["open"]) <= -3:
                data[2] = -3 # 손절
                break
                
            elif get_rate(job_coin[idx][j]["high"], job_coin[idx][1440 * i]["open"]) > 10:
                check = "half" 
            
            elif check == "half" and get_rate(job_coin[idx][j]["low"], job_coin[idx][1440 * i]["open"]) <= 0:
                data[2] = 5  # 경우 2
                break
            
            elif check == "half" and get_rate(job_coin[idx][j]["high"], job_coin[idx][1440 * i]["open"]) > 50:
                data[2] = 37 # 경우 1
                break
        
        if check == "half": # 경우 3
            data[2] = 5 + 0.55 * get_rate(job_coin[idx][1439 + (1440*idx)]["close"], job_coin[idx][1440 * i]["open"])
        else: # 경우 4
            data[2] = get_rate(job_coin[idx][1439 + (1440*idx)]["close"], job_coin[coin_cnt]["open"][0])
        coin_cnt += 1          
        coin_analysis.append(data)
            

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
fff

In [ ]:
data[1]

In [4]:
sss = data

In [3]:
ddd = data

In [24]:
coins = pyupbit.get_tickers(fiat="KRW")
data = []
dates = date_range("2021-09-11", yesterday)
coin_data = []
for idx in notebook.tqdm(range(1, len(dates) - 2)):
    coins = get_pick_coins(dates[idx])
    coin_data.append(coins)
    row = []
    for coin in coins:
        minute_data = pyupbit.get_ohlcv(coin[1], to=f"{dates[idx + 2]} 09:00:00", interval="minute1", count=1440)
        time.sleep(1)
        row.append(minute_data)
        
    data.append(row)
    

In [ ]:
coins_rate.append([date, coin, coin_rate, ""])

In [ ]:
coins = get_pick_coins("2021-09-12")

In [ ]:
coin_data = pyupbit.get_ohlcv("KRW-XRP", to="2021-09-14 09:00:00", interval="minute1", count=1440)
coin_data

In [ ]:
coins

In [ ]:
max(coin_data.high)
min(coin_data.low)

In [ ]:
# 9월 12

In [ ]:
coin = pyupbit.get_ohlcv("KRW-XRP", to="2021-09-13", count=1)

In [ ]:
coin_data = pd.DataFrame(coin_data)
coin_data = coin_data.values.tolist()

In [ ]:
coin_data

In [ ]:
ddd[0][0][1]

In [ ]:
24 * 60 * 60

In [12]:
len(data[0])

1440

In [14]:
1440 * 5

7200

In [27]:
len(data[0][0])

1440